# Testing DRPredICT Modules

**Authorship:**
Adam Klie, *03/29/2022*
***
**Description:**
Notebook for testing out modules for building DRPredICT architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

In [18]:
import numpy as np
import pandas as pd
import torch
# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import sys
sys.path.append("../drpredict")

# Random Dataset

In [2]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

## VanillaAE

In [16]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)
training_dataset[0][0].shape

torch.Size([1000])

In [19]:
from VanillaAE import VanillaAE

In [23]:
encoder_args = {"activation": "sigmoid"}
decoder_args = {"dropout_rate": 0.2}

In [24]:
mutAE = VanillaAE(1000, 10, hidden_dims=[500], encoder_kwargs=encoder_args, decoder_kwargs=decoder_args)

In [29]:
for i_batch, batch in enumerate(training_dataloader):
    x = batch[0]
    outs = mutAE(x)
    print(x.shape, outs.shape)
    if i_batch==3:
        break

torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])


# Multiomic

In [ ]:
dataset.Da

In [59]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000))

In [46]:
omic_types = ["mutations", "expression", "cn"]
in_dims = [1000, 1000, 1000]
out_dims = [6, 6, 6]
h_dims = [[500], [500], [500]]
encoder_args = [{"activation": "sigmoid"}, {"activation": "relu"}, {"activation": "sigmoid"}]
fcn_args = {"dropout_rate": 0.2, "output_dim": 1}

In [47]:
from MultiEncoder import MultiEncoder

In [49]:
drpredict = MultiEncoder(omics=omic_types, input_dims=in_dims, output_dims=out_dims, hidden_dims=h_dims, encoder_kwargs=encoder_args, fcn_kwargs=fcn_args)

In [60]:
for i, omic in enumerate(drpredict.encoders.keys()):
    print(i, drpredict.encoders[omic])

0 FullyConnectedModule(
  (module): Sequential(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=500, out_features=6, bias=True)
  )
)
1 FullyConnectedModule(
  (module): Sequential(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=6, bias=True)
  )
)
2 FullyConnectedModule(
  (module): Sequential(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=500, out_features=6, bias=True)
  )
)
